In [134]:
from cmath import sqrt
import numpy as np
ket0 = np.array([1,0]) # |0>
ket1 = np.array([0,1])  # |1>
ketPlus = (ket0 + ket1)/np.sqrt(2) # |+>
ketMinus = (ket0 - ket1)/np.sqrt(2) # |->
H = 1/np.sqrt(2)*np.array([[1,1],[1,-1]]) # Hadamard gate

# Pauli matrices are a set of three 2x2 real matrices used in quantum mechanics to represent the three Pauli operators (X, Y, and Z) and the identity operator (I). 
# The Pauli matrices are denoted by σ1, σ2, and σ3, respectively, and are given by the following expressions:
# σ1 = [[0,1],[1,0]]    σ2 = [[0,-i],[i,0]]    σ3 = [[1,0],[0,-1]]   I = [[1,0],[0,1]] 
# The Pauli matrices are the generators of the Pauli group, which is the group of all unitary matrices of the form e^{iθP} where P is a Pauli matrix and θ is a real number.
# Hemitian, involutory, and unitary matrices are all Pauli matrices.

X = np.array([[0,1],[1,0]]) # Pauli-X gate, also known as NOT gate
Y = np.array([[0,-1j],[1j,0]]) # Pauli-Y gate, also known as NOT gate
Z = np.array([[1,0],[0,-1]]) # Pauli-Z gate, also known as phase flip gate

SQRT_X = np.array([[1+1j,1-1j],[1-1j,1+1j]])/2 # Square root of X gate, pi phase shift gate
S = np.array([[1,0],[0,1j]]) # Phase gate,also know as sqrt(Z) gate, pi/2 phase shift gate
T = np.array([[1,0],[0,np.exp(1j*np.pi/4)]]) # T gate, also known as sqrt(S) gate, or root4(Z) gate, or pi/4 phase shift gate

SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) # SWAP gate (exchange gate) |a,b> -> |b,a>
SQRT_SWAP = np.array([[1,0,0,0],[0,(1+1j)/2,(1-1j)/2,0],[0,(1-1j)/2,(1+1j)/2,0],[0,0,0,1]]) # Square root of SWAP gate,

# Controlled gates
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]]) # Controlled NOT gate, also known as CNOT gate, or controlled X gate, |a,b> -> |a,b XOR a> 
CZ = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]]) # Controlled Z gate, also known as controlled phase flip gate 
CCNOT = np.array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]]) # Toffoli gate, also known as CCNOT gate, or controlled controlled NOT gate


In [135]:
def phase_shift_gate(theta):
    return np.array([[1,0],[0,np.exp(1j*theta)]]) # Phase shift gate 

In [136]:
H @ ket0 # H|0> = |+>
H @ ket1 # H|1> = |->

array([ 0.70710678, -0.70710678])

In [137]:
from abc import ABCMeta, abstractmethod
from contextlib import contextmanager

In [138]:
class Qubit(metaclass=ABCMeta):
    @abstractmethod
    def h(self):
        pass

    @abstractmethod
    def measure(self) -> bool:
        pass

    @abstractmethod
    def reset(self):
        pass

In [139]:
class QuantumDevice(metaclass=ABCMeta):
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        pass
    
    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        pass

    @contextmanager
    def using_qubit(self):
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

In [140]:
KET_0 = np.array([1, 0]) # |0>
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2) # Hadamard gate

In [141]:
class SimulatedQubit(Qubit):
    def _init_(self):
        self.reset()

    def h(self):
        self.state = H @ self.state

    def measure(self) -> bool:
        prob_0 = np.abs(self.state[0]) ** 2
        sample = np.random.random() <= prob_0
        return bool(0 if sample else 1)

    def reset(self):
        self.state = KET_0.copy()

In [142]:
class SingleQubitSimulator(QuantumDevice):
    available_qubits : list[SimulatedQubit] = [SimulatedQubit()]

    def allocate_qubit(self) -> SimulatedQubit:
        if self.available_qubits:
            return self.available_qubits.pop()

    def deallocate_qubit(self, qubit: SimulatedQubit):
        self.available_qubits.append(qubit)

In [143]:
def qrng(device: QuantumDevice) -> bool:
    with device.using_qubit() as q:
        q.h()
        return q.measure()

In [145]:
qsim = SingleQubitSimulator()
for idx_sample in range(10):
    random_sample = qrng(qsim)
    print(f"Our QRNG returned {random_sample}.")

Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned True.
Our QRNG returned True.
